# Лабораторная работа №4


## Задание

Провести классификацию найденного датасета, методами наивного Байеса

Импорт библиотек

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.naive_bayes import GaussianNB

Загрузка датасета

In [2]:
df = pd.read_csv('weather.csv', encoding='utf-8')
df['RainToday']= df['RainToday'].map({'Yes': True, 'No': False}).astype(bool) 
df['RainTomorrow']= df['RainTomorrow'].map({'Yes': True, 'No': False}).astype(bool)
print(df.head(5))
print(df.dtypes)

         Date Location  MinTemp  MaxTemp  Rainfall  Evaporation  Sunshine  \
0  2008-12-01   Albury     13.4     22.9       0.6          NaN       NaN   
1  2008-12-02   Albury      7.4     25.1       0.0          NaN       NaN   
2  2008-12-03   Albury     12.9     25.7       0.0          NaN       NaN   
3  2008-12-04   Albury      9.2     28.0       0.0          NaN       NaN   
4  2008-12-05   Albury     17.5     32.3       1.0          NaN       NaN   

  WindGustDir  WindGustSpeed WindDir9am  ... Humidity9am  Humidity3pm  \
0           W           44.0          W  ...        71.0         22.0   
1         WNW           44.0        NNW  ...        44.0         25.0   
2         WSW           46.0          W  ...        38.0         30.0   
3          NE           24.0         SE  ...        45.0         16.0   
4           W           41.0        ENE  ...        82.0         33.0   

   Pressure9am  Pressure3pm  Cloud9am  Cloud3pm  Temp9am  Temp3pm  RainToday  \
0       1007.7    

Удаление ненужных столбцов из датасета

In [3]:
columns_to_drop = ['Humidity3pm', 'Pressure3pm', 'Cloud3pm', 'Temp3pm', 'WindDir9am']
df = df.drop(columns=columns_to_drop, axis=1)
print(df.head(5))

         Date Location  MinTemp  MaxTemp  Rainfall  Evaporation  Sunshine  \
0  2008-12-01   Albury     13.4     22.9       0.6          NaN       NaN   
1  2008-12-02   Albury      7.4     25.1       0.0          NaN       NaN   
2  2008-12-03   Albury     12.9     25.7       0.0          NaN       NaN   
3  2008-12-04   Albury      9.2     28.0       0.0          NaN       NaN   
4  2008-12-05   Albury     17.5     32.3       1.0          NaN       NaN   

  WindGustDir  WindGustSpeed WindDir3pm  WindSpeed9am  WindSpeed3pm  \
0           W           44.0        WNW          20.0          24.0   
1         WNW           44.0        WSW           4.0          22.0   
2         WSW           46.0        WSW          19.0          26.0   
3          NE           24.0          E          11.0           9.0   
4           W           41.0         NW           7.0          20.0   

   Humidity9am  Pressure9am  Cloud9am  Temp9am  RainToday  RainTomorrow  
0         71.0       1007.7       8.

Удаление строк с пропущенными значениями

In [4]:
df.dropna(inplace=True)

Предобработка данных

In [5]:
# Кодирование категориальных признаков
label_encoders = {}
for column in df.select_dtypes(include=['object']).columns:
    label_encoders[column] = LabelEncoder()
    df[column] = label_encoders[column].fit_transform(df[column])
    
print(df.head(5))

      Date  Location  MinTemp  MaxTemp  Rainfall  Evaporation  Sunshine  \
6049   418         4     17.9     35.2       0.0         12.0      12.3   
6050   419         4     18.4     28.9       0.0         14.8      13.0   
6052   421         4     19.4     37.6       0.0         10.8      10.6   
6053   422         4     21.9     38.4       0.0         11.4      12.2   
6054   423         4     24.2     41.0       0.0         11.2       8.4   

      WindGustDir  WindGustSpeed  WindDir3pm  WindSpeed9am  WindSpeed3pm  \
6049           11           48.0          12           6.0          20.0   
6050            8           37.0          10          19.0          19.0   
6052            5           46.0           6          30.0          15.0   
6053           14           31.0          15           6.0           6.0   
6054           14           35.0          14          17.0          13.0   

      Humidity9am  Pressure9am  Cloud9am  Temp9am  RainToday  RainTomorrow  
6049         20

In [6]:
# Масштабирование числовых признаков
scaler = StandardScaler()
numeric_features = df.select_dtypes(include=['int32', 'int64', 'float32', 'float64']).columns
df[numeric_features] = scaler.fit_transform(df[numeric_features])

print(df.head(5))

          Date  Location   MinTemp   MaxTemp  Rainfall  Evaporation  Sunshine  \
6049 -1.545272 -1.182653  0.716750  1.589217 -0.302062     1.771778  1.217662   
6050 -1.544105 -1.182653  0.793979  0.686505 -0.302062     2.529392  1.404005   
6052 -1.541771 -1.182653  0.948437  1.933107 -0.302062     1.447086  0.765115   
6053 -1.540604 -1.182653  1.334582  2.047737 -0.302062     1.609432  1.191042   
6054 -1.539437 -1.182653  1.689834  2.420285 -0.302062     1.555317  0.179465   

      WindGustDir  WindGustSpeed  WindDir3pm  WindSpeed9am  WindSpeed3pm  \
6049     0.728337       0.553696    0.933445     -1.077916      0.043157   
6050     0.102162      -0.269995    0.509257      0.435573     -0.074484   
6052    -0.524013       0.403934   -0.339118      1.716217     -0.545047   
6053     1.354511      -0.719280    1.569726     -1.077916     -1.603814   
6054     1.354511      -0.419757    1.357632      0.202728     -0.780329   

      Humidity9am  Pressure9am  Cloud9am   Temp9am  Rain

Разделение данных на обучающий и тестовый наборы

In [7]:
X = df.drop('RainTomorrow', axis=1)
Y = df['RainTomorrow']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

##### Модель наивного Байеса

Создание и обучение модели наивного Байеса

In [8]:
model = GaussianNB()

In [9]:
model.fit(X_train, Y_train)

GaussianNB()

Предсказание на тестовом наборе данных

In [10]:
Y_pred = model.predict(X_test)

Оценка модели GaussianNB

In [11]:
report = classification_report(Y_test, Y_pred)
print(report)

              precision    recall  f1-score   support

       False       0.88      0.87      0.88      9326
        True       0.56      0.57      0.56      2589

    accuracy                           0.81     11915
   macro avg       0.72      0.72      0.72     11915
weighted avg       0.81      0.81      0.81     11915



Оценка модели на тестовом наборе данных

In [12]:
accuracy = accuracy_score(Y_test, Y_pred)
print("Оценка модели:", accuracy)

Оценка модели: 0.8081409987410827
